In [1]:
import numpy as np
import math

## A.1 日時

- 日付`NDay`は1/1からの通しの日数
- 時刻`NHour`は整数, 0時～23時が基本。12/31のみ24時あり
- 時刻`TT`は、時間分割`MM`における時刻
  - `MM`：1時間の内の$(1/$ `NDT`$)$間隔の順番, 正時が`MM`$=0$, `MM`$=0～$ `NDT`$-1$


- 時刻`Hour01`は、前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の時間分割を、その正時に属するものとして扱うための時刻 → 時間毎の効果係数を算定するために使用するだけ

In [2]:
def calc_NDayNHour(Hour00):
    if Hour00==8760:
        [NDay, NHour] = [365, 24]
    else:
        NDay = Hour00//24 + 1
        NHour = Hour00 - (NDay - 1) * 24 

    return [NDay,NHour]

In [3]:
def calc_TT(NHour, NDT, MM):
    return NHour + MM / float(NDT)

In [4]:
def calc_Hour01(TT):
    return int(TT + 0.5)

## A.2 赤緯の計算 (仕様書6.2 式(4))

$ \begin{align}
\delta_d = (180 / \pi) & \{0.006322 - 0.405748 \cos (2 \pi N / 366 + 0.153231)\\
& - 0.005880 \cos (4 \pi N / 366 + 0.207099)\\
& - 0.003233 \cos (6 \pi N / 366 + 0.620129) \}
\end{align} $

$\delta_d$: 赤緯, deg  
$N$: 1月1日を$N=1$とした年頭からの通しの日数, d  

右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

In [5]:
def calc_deltad(NDay):
    return (180 / math.pi) * (0.006322 - 0.405748 * math.cos(2 * math.pi * float(NDay) / 366 + 0.153231)
                                       - 0.005880 * math.cos(4 * math.pi * float(NDay) / 366 + 0.207099)
                                       - 0.003233 * math.cos(6 * math.pi * float(NDay) / 366 + 0.620129))

## A.3 均時差の計算 (仕様書6.2 式(6))

$ \begin{align}
e_d = -0.000279 &+ 0.122772 \cos (2 \pi N / 366 + 1.498311)\\
& - 0.165458 \cos (4 \pi N / 366 - 1.261546)\\
& - 0.005354 \cos (6 \pi N / 366 - 1.1571) \}
\end{align} $

$e_d$: 均時差, h  
$N$: 1月1日を$N=1$とした年頭からの通しの日数, d

右辺の余弦のかっこ内の角度は$radian$単位となっているので注意

In [6]:
def calc_eed(NDay):
    return ( -0.000279 + 0.122772 * math.cos(2 * math.pi * NDay / 366 + 1.498311)
                       - 0.165458 * math.cos(4 * math.pi * NDay / 366 - 1.261546)
                       - 0.005354 * math.cos(6 * math.pi * NDay / 366 - 1.1571)   )

## A.4 時角の計算 (仕様書6.2 式(7))

$T_{d,t} = (t + e_d - 12) \times 15 + (L - 135) $

$T_{d,t}$: 時角, deg  
$t$: 時刻, h  
$e_d$: 均時差, h  
$L$: 経度, deg

In [7]:
def calc_Tdt(Longitude, eed, TT):
    return ( TT + eed - 12) * 15 +(Longitude - 135)

### 太陽高度の計算

#### Outline

$$
\displaystyle
\sin h_S = \max(0, \sin \phi \sin \delta + \cos \phi \cos \delta \cos T)
$$

$$
\displaystyle
\cos h_S = (1 - \sin ^2 h_S)^{0.5}
$$

$$
\displaystyle
h_S = \sin^{-1} (\sin h_S)
$$

$h_S$ is solar altitude / 太陽高度, rad  
$\phi$ is latitude, 緯度, rad  
$\delta$ is declination / 赤緯, rad   
$T$ is hour angle / 時角, rad

#### Function

In [8]:
def calc_hs(latitude, delta, t):

    sin_hs = max(0.0,
               math.sin(latitude) * math.sin(delta) \
               + math.cos(latitude) * math.cos(delta) * math.cos(t)
              )
    
    cos_hs = (1 - sin_hs**2) **0.5
    
    if sin_hs == 1.0:
        hs = math.radians(90.0)
    else:
        hs = math.atan2(sin_hs, cos_hs)
    
    return hs, sin_hs, cos_hs

## A.7 太陽方位角の計算 (仕様書6.2 式(10)～(12))

- 太陽方位角$A_{ZS,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 赤緯$\delta_d[deg]$, 時角$T_{d,t}[deg]$, 緯度$\phi[deg]$

$$\sin A_{ZS,d,t} = \cos \delta_d \sin T_{d,t} / \cos h_{S,d,t} \qquad (10) $$

$$\cos A_{ZS,d,t} = (\sin h_{S,d,t} \sin \phi - \sin \delta_d) / (\cos h_{S,d,t} \cos \phi) \qquad (11) $$

$$
A_{ZS,d,t} = \left\{
\begin{array}{ll}
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) + 180 \hspace{24pt} (\sin A_{ZS,d,t} > 0, \cos A_{ZS,d,t} < 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) - 180 \hspace{24pt} (\sin A_{ZS,d,t} < 0, \cos A_{ZS,d,t} < 0)
\\
90 \hspace{136pt} (\sin A_{ZS,d,t} = 1, \cos A_{ZS,d,t} = 0)
\\
-90 \hspace{130pt} (\sin A_{ZS,d,t} = -1, \cos A_{ZS,d,t} = 0)
\\
\tan^{-1} (\sin A_{ZS,d,t} / \cos A_{ZS,d,t}) \hspace{48pt} (other)
\end{array}
\right.  \qquad (12) 
$$

In [9]:
def calc_Azsdt(Latitude, deltad, Tdt, sinh, cosh):
    
#    sinAzsdt = np.cos(np.radians(deltad)) * np.sin(np.radians(Tdt)) / cosh
#    cosAzsdt = ( ( sinh * np.sin(np.radians(Latitude)) - np.sin(np.radians(deltad)) ) 
#             /   ( cosh * np.cos(np.radians(Latitude)) ) )
#    if abs(sinAzsdt) == 1:
#        Azsdt = 90 * sinAzsdt
#    elif sinAzsdt > 0 and cosAzsdt < 0:
#        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt )) + 180
#    elif sinAzsdt < 0 and cosAzsdt < 0:        
#        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt) ) - 180   
#    else:
#        Azsdt = np.rad2deg(np.arctan( sinAzsdt / cosAzsdt) )
   
    sinAzsdt = math.cos(math.radians(deltad)) * math.sin(math.radians(Tdt)) / cosh
    cosAzsdt = ( ( sinh * math.sin(math.radians(Latitude)) - math.sin(math.radians(deltad)) ) 
             /   ( cosh * math.cos(math.radians(Latitude)) ) )
    if abs(sinAzsdt) == 1:
        return 90 * sinAzsdt
    elif sinAzsdt > 0 and cosAzsdt < 0:
        return math.degrees(math.atan( sinAzsdt / cosAzsdt )) + 180
    elif sinAzsdt < 0 and cosAzsdt < 0:        
        return math.degrees(math.atan( sinAzsdt / cosAzsdt) ) - 180   
    else:
        return math.degrees(math.atan( sinAzsdt / cosAzsdt) )


## A.8 窓面の方位 (仕様書5.2 図4)

- 窓面の方位は、以下の通り
  - 北北東：$-157.5°$, 北東：$-135°$, …, 東：$-90°$, …, 南：$0°$, …, 西：$+90°$, …,北：$+180°$
  - 角度指定も可：$-180°< A_{ZW,j} \leq +180°$
  - デフォルトは8方位指定

In [10]:
def calc_Azwj(Azimuth):
    
    Azimuth00 = ["北北東", "北東", "東北東", "東", "東南東", "南東", "南南東", "南"
                 , "南南西", "南西", "西南西", "西", "西北西", "北西", "北北西", "北" ]
    if Azimuth in Azimuth00:
        return (Azimuth00.index(Azimuth) - 7) * 22.5
    elif -180 < float(Azimuth) <= 180:
        return float(Azimuth) 
    else:
        raise ValueError('窓面方位の入力が不適切です')

## A.9 窓面の法線ベクトルと太陽位置とのなす水平面上の角度の計算 (仕様書6.2 式(1))

- 窓面の法線ベクトルと太陽位置とのなす水平面上の角度$A_{ZW,j,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,j}[deg]$

$$
A_{ZW,j,d,t} = \left\{
\begin{array}{ll}
A_{ZS,d,t} - A_{ZW,j} \hspace{48pt} (-180 < A_{ZS,d,t} - A_{ZW,j} \leq 180)
\\
A_{ZS,d,t} - A_{ZW,j} + 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \leq -180)
\\
A_{ZS,d,t} - A_{ZW,j} - 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \geq 180)
\end{array}
\right.  \qquad (1) 
$$

In [11]:
def calc_Azwjdt(Azwj, Azsdt):
    
    Azwjdt = Azsdt - Azwj
    if Azwjdt < -180:
        return Azwjdt + 360.0
    elif Azwjdt > 180:
        return Azwjdt - 360.0
    else:
        return Azwjdt

## A.10 窓まわり寸法のデータの持たせ方デフォルト (仕様書5.1 図2)

In [12]:
def set_WSSize(WSSize1):
    
    WSSizeDict = {}
    
    for i in range(0, len(WSSize1), 2):
        WSSizeDict[WSSize1[i]] = WSSize1[i+1]

    # X1
    if "X1" not in WSSizeDict:
        WSSizeDict["X1"] = 0
    elif WSSizeDict["X1"] < 0:
        raise ValueError("寸法X1の設定が不適切です")
    elif WSSizeDict["X1"] == "":
        WSSizeDict["X1"] = 0        
        
    # X2
    if "X2" not in WSSizeDict:
        raise ValueError("寸法X2が設定されていません")
    elif WSSizeDict["X2"] <= 0 or WSSizeDict["X2"] == "":
        raise ValueError("寸法X2の設定が不適切です")            
        
    # X3
    if "X3" not in WSSizeDict:
        WSSizeDict["X3"] = 0
    elif WSSizeDict["X3"] < 0:
        raise ValueError("寸法X3の設定が不適切です")
    elif WSSizeDict["X3"] == "":
        WSSizeDict["X3"] = 0 
        
    # Y1
    if "Y1" not in WSSizeDict:
        WSSizeDict["Y1"] = 0
    elif WSSizeDict["Y1"] < 0:
        raise ValueError("寸法Y1の設定が不適切です")
    elif WSSizeDict["Y1"] == "":
        WSSizeDict["Y1"] = 0 
        
    # Y2
    if "Y2" not in WSSizeDict:
        raise ValueError("寸法Y2が設定されていません")
    elif WSSizeDict["Y2"] <= 0 or WSSizeDict["Y2"] == "":
        raise ValueError("寸法Y2の設定が不適切です")            

    # Y3
    if "Y3" not in WSSizeDict:
        WSSizeDict["Y3"] = 0
    elif WSSizeDict["Y3"] < 0:
        raise ValueError("寸法Y3の設定が不適切です")            
    elif WSSizeDict["Y3"] == "":
        WSSizeDict["Y3"] = 0 
        
    # Zxp
    if "Zxp" not in WSSizeDict:
        WSSizeDict["Zxp"] = 0
    elif WSSizeDict["Zxp"] < 0:
        raise ValueError("寸法Zxpの設定が不適切です")
    elif WSSizeDict["Zxp"] == "":
        WSSizeDict["Zxp"] = 0 
        
    # Zxm
    if "Zxm" not in WSSizeDict:
        WSSizeDict["Zxm"] = 0
    elif WSSizeDict["Zxm"] < 0:
        raise ValueError("寸法Zxmの設定が不適切です")           
    elif WSSizeDict["Zxm"] == "":
        WSSizeDict["Zxm"] = 0 
        
    # Zyp
    if "Zyp" not in WSSizeDict:
        WSSizeDict["Zyp"] = 0
    elif WSSizeDict["Zyp"] < 0:
        raise ValueError("寸法Zypの設定が不適切です")     
    elif WSSizeDict["Zyp"] == "":
        WSSizeDict["Zyp"] = 0 
        
    # Zym
    if "Zym" not in WSSizeDict:
        WSSizeDict["Zym"] = 0
    elif WSSizeDict["Zym"] < 0:
        raise ValueError("寸法Zymの設定が不適切です")
    elif WSSizeDict["Zym"] == "":
        WSSizeDict["Zym"] = 0 
        
    """ 以下、オプション扱い → 非入力は窓端から日よけの付け根までの距離とする """     
    # X1yp
    if "X1yp" not in WSSizeDict:
        WSSizeDict["X1yp"] = WSSizeDict["X1"] 
    elif WSSizeDict["X1yp"] < 0:
        raise ValueError("寸法X1ypの設定が不適切です")        
    elif WSSizeDict["X1yp"] > WSSizeDict["X1"] or WSSizeDict["X1yp"] == "":
        WSSizeDict["X1yp"] = WSSizeDict["X1"]         
        
    # X1ym
    if "X1ym" not in WSSizeDict:
        WSSizeDict["X1ym"] = WSSizeDict["X1"] 
    elif WSSizeDict["X1ym"] < 0:
        raise ValueError("寸法X1ymの設定が不適切です")
    elif WSSizeDict["X1ym"] > WSSizeDict["X1"] or WSSizeDict["X1ym"] == "":
        WSSizeDict["X1ym"] = WSSizeDict["X1"]       
        
    # X3yp
    if "X3yp" not in WSSizeDict:
        WSSizeDict["X3yp"] = WSSizeDict["X3"] 
    elif WSSizeDict["X3yp"] < 0:
        raise ValueError("寸法X3ypの設定が不適切です")            
    elif WSSizeDict["X3yp"] > WSSizeDict["X3"] or WSSizeDict["X3yp"] == "":
        WSSizeDict["X3yp"] = WSSizeDict["X3"]   
        
    # X3ym
    if "X3ym" not in WSSizeDict:
        WSSizeDict["X3ym"] = WSSizeDict["X3"] 
    elif WSSizeDict["X3ym"] < 0:
        raise ValueError("寸法X3ymの設定が不適切です")               
    elif WSSizeDict["X3ym"] > WSSizeDict["X3"] or WSSizeDict["X3ym"] == "":
        WSSizeDict["X3ym"] = WSSizeDict["X3"]   
        
    # Y1xp
    if "Y1xp" not in WSSizeDict:
        WSSizeDict["Y1xp"] = WSSizeDict["Y1"]
    elif WSSizeDict["Y1xp"] < 0:
        raise ValueError("寸法Y1xpの設定が不適切です")     
    elif WSSizeDict["Y1xp"] > WSSizeDict["Y1"] or WSSizeDict["Y1xp"] == "":
        WSSizeDict["Y1xp"] = WSSizeDict["Y1"]  
        
    # Y1xm
    if "Y1xm" not in WSSizeDict:
        WSSizeDict["Y1xm"] = WSSizeDict["Y1"]  
    elif WSSizeDict["Y1xm"] < 0:
        raise ValueError("寸法Y1xmの設定が不適切です")                
    elif WSSizeDict["Y1xm"] > WSSizeDict["Y1"] or WSSizeDict["Y1xm"] == "":
        WSSizeDict["Y1xm"] = WSSizeDict["Y1"]  
        
    # Y3xp
    if "Y3xp" not in WSSizeDict:
        WSSizeDict["Y3xp"] = WSSizeDict["Y3"]
    elif WSSizeDict["Y3xp"] < 0:
        raise ValueError("寸法Y3xpの設定が不適切です")
    elif WSSizeDict["Y3xp"] > WSSizeDict["Y3"] or WSSizeDict["Y3xp"] == "":
        WSSizeDict["Y3xp"] = WSSizeDict["Y3"]
        
    # Y3xm
    if "Y3xm" not in WSSizeDict:
        WSSizeDict["Y3xm"] = WSSizeDict["Y3"]
    elif WSSizeDict["Y3xm"] < 0:
        raise ValueError("寸法Y3xmの設定が不適切です")            
    elif WSSizeDict["Y3xm"] > WSSizeDict["Y3"] or WSSizeDict["Y3xm"] == "":
        WSSizeDict["Y3xm"] = WSSizeDict["Y3"]
        
    WSSize = [WSSizeDict["X1"],  WSSizeDict["X2"],  WSSizeDict["X3"],
              WSSizeDict["X1yp"],WSSizeDict["X1ym"],WSSizeDict["X3yp"],WSSizeDict["X3ym"],
              WSSizeDict["Y1"],  WSSizeDict["Y2"],  WSSizeDict["Y3"],
              WSSizeDict["Y1xp"],WSSizeDict["Y1xm"],WSSizeDict["Y3xp"],WSSizeDict["Y3xm"],
              WSSizeDict["Zxp"], WSSizeDict["Zxm"], WSSizeDict["Zyp"], WSSizeDict["Zym"] ]
            
    return WSSize